In [1]:
import csv
import numpy as np
import random
from collections import defaultdict

query_id_relevant_doc_id = defaultdict(list)
with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\qrels.dev.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
        query_id = int(row[0])
        relevant_doc_id = int(row[2])
        query_id_relevant_doc_id[query_id].append(relevant_doc_id)
        
print (len(query_id_relevant_doc_id))


6986


In [3]:
query_id_relevant_doc_id.keys()

dict_keys([300674, 125705, 94798, 9083, 174249, 320792, 1090270, 1101279, 201376, 54544, 118457, 178627, 1101278, 68095, 87892, 257309, 1090242, 211691, 165002, 1101276, 264827, 342285, 372586, 89786, 118448, 92542, 206117, 141472, 293992, 196232, 352818, 45924, 208145, 79891, 208494, 319564, 155234, 14151, 67802, 1090184, 323382, 323998, 91711, 125898, 289812, 333486, 1090171, 73257, 1090170, 237373, 127876, 85095, 1090165, 259417, 1101271, 281930, 205107, 307118, 87019, 335710, 127984, 1090151, 46711, 1090146, 1090132, 1090115, 1090110, 1090107, 1090102, 1090100, 1090086, 1090077, 1090072, 1090063, 1090054, 1090043, 1101259, 1090029, 1089983, 1089966, 1089964, 1089945, 1089940, 1089925, 1089906, 1089896, 1101236, 1089868, 1089846, 1089832, 1089810, 1101228, 1089805, 1089804, 1089787, 1089776, 1089763, 1089760, 1089750, 1089727, 1089719, 1089706, 1089696, 1089693, 1089691, 1089688, 1089683, 1089678, 1089674, 1089670, 1089656, 1101214, 1089645, 1089639, 1101211, 1089619, 1089597, 10895

In [8]:
query_id_sample_id = defaultdict(list)
count = 0

fr = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\top1000.dev.tsv", encoding="UTF8")
rd = csv.reader(fr, delimiter='\t')
query_id_sample_id = defaultdict(list)
for row in rd:
    query_id = int(row[0])
    doc_id = int(row[1])
    if query_id_relevant_doc_id.get(query_id) != None and doc_id not in query_id_relevant_doc_id.get(query_id):
        query_id_sample_id[query_id].append(doc_id)
    count += 1
    if count % 100000 == 0:
        print(count)

print (len(query_id_sample_id))

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6979


In [14]:
fw = open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\RandomNegDocSamplesDev.tsv", "w" )
for query_id, doc_ids in query_id_sample_id.items():
    fw.write(str(query_id) + "\t" + ",".join(str(x) for x in doc_ids) + "\n")

In [15]:
import pickle

with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\model\\bm25.pkl", 'rb') as fr:
    bm25 = pickle.load(fr)

C:\Users\mapyredd\AppData\Local\Continuum\anaconda2\envs\deeplearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
import sys
from importlib import reload
import logging
import csv
from nltk.stem.porter import PorterStemmer
import string
import gensim
import pickle
from gensim.summarization.bm25 import BM25


porterStemmer = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def removeNonAscii(s):
    return "".join(filter(lambda x:ord(x) < 128, s))


def preprocess(passage):
    clean_passage = passage.lower()
    clean_passage = removeNonAscii(clean_passage)
    clean_passage = clean_passage.translate(str.maketrans('', '', string.punctuation)) 
    
    clean_passage_list = clean_passage.split()
    clean_passage_list_stem = []
    for word in clean_passage_list:
        if word not in gensim.parsing.preprocessing.STOPWORDS:
            clean_passage_list_stem.append(porterStemmer.stem(word))
    return clean_passage_list_stem


In [17]:
import csv
from collections import defaultdict


average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

query_file = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\queries.dev.tsv",encoding='utf-8')
query_reader = csv.reader(query_file, delimiter = "\t")

queryid_text_dict = defaultdict(str)
for row in query_reader:
    queryid_text_dict[int(row[0])] = preprocess(row[1])
    
print (len(queryid_text_dict))

12665


In [18]:
sample_doc_ids = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\RandomNegDocSamplesDev.tsv")
sample_reader = csv.reader(sample_doc_ids, delimiter='\t')

count = 0
qrel_nonrelevant_file = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\qrels_nr.dev.tsv",'w')
for row in sample_reader:
    query_id = int(row[0])
    docid_scores = []
    query_text = queryid_text_dict[query_id]
    doc_ids = row[1].split(',')
    for doc_id in doc_ids:
        score = bm25.get_score(query_text, int(doc_id), average_idf)
        docid_scores.append((int(doc_id), score))
    docid_scores = sorted(docid_scores, key=lambda x:x[1], reverse=True)[:15]

    for doc_score in docid_scores:
        qrel_nonrelevant_file.write(str(query_id) + "\t" + "0" + "\t" + str(doc_score[0]) + "\t" + "0" + "\n")
    count += 1
    if count %1000 == 0:
        print (count)

1000
2000
3000
4000
5000
6000
